# Setup Comprehend Through AWS Console 

<img src="img/comprehend.png" width="80%" align="left">

In [1]:
%store -r

In [2]:
import sys
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
role = sagemaker.get_execution_role()

sm = boto3.Session().client(service_name='sagemaker', region_name=region_name)

In [3]:
comprehend = boto3.client('comprehend')

In [4]:
print(noheader_train_s3_uri)

s3://sagemaker-us-east-1-322537213286/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv


In [5]:
!aws s3 ls $noheader_train_s3_uri

2020-11-16 12:06:49   29953816 amazon_reviews_us_Digital_Software_v1_00_noheader.csv


# Create Data Access Role for Comprehend

## Create Policy

In [6]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "comprehend.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
} 

## Create Role and Attach Policies

In [7]:
iam_comprehend_role_name = 'aws_sm_hol_Comprehend'

In [8]:
import json
import boto3
import time

from botocore.exceptions import ClientError

try:
    iam = boto3.client('iam')

    iam_role_comprehend = iam.create_role(
        RoleName=iam_comprehend_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='SageMAker HOL Comprehend Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        iam_role_comprehend = iam.get_role(RoleName=iam_comprehend_role_name)
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)
        
time.sleep(30)

Role already exists


In [9]:
comprehend_s3_policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "s3:GetObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(data_bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(data_bucket)
            ],
            "Effect": "Allow"
        },
        {
            "Action": [
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::{}/*".format(job_bucket)
            ],
            "Effect": "Allow"
        }
    ]
}

print(comprehend_s3_policy_doc)

{'Version': '2012-10-17', 'Statement': [{'Action': ['s3:GetObject'], 'Resource': ['arn:aws:s3:::sagemaker-us-east-1-322537213286/*'], 'Effect': 'Allow'}, {'Action': ['s3:ListBucket'], 'Resource': ['arn:aws:s3:::sagemaker-us-east-1-322537213286'], 'Effect': 'Allow'}, {'Action': ['s3:PutObject'], 'Resource': ['arn:aws:s3:::sagemaker-experiments-us-east-1-322537213286/*'], 'Effect': 'Allow'}]}


# Attach Policy to Role

In [10]:
import time

response = iam.put_role_policy(
    RoleName=iam_comprehend_role_name,
    PolicyName='AWS_SM_HOL_ComprehendPolicyToS3',
    PolicyDocument=json.dumps(comprehend_s3_policy_doc)
)

print(response)

time.sleep(30)

{'ResponseMetadata': {'RequestId': '0a744c1d-62a7-4ea1-aeb4-3ef73b5199d1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0a744c1d-62a7-4ea1-aeb4-3ef73b5199d1', 'content-type': 'text/xml', 'content-length': '206', 'date': 'Mon, 16 Nov 2020 12:07:42 GMT'}, 'RetryAttempts': 0}}


# Train the Model

In [11]:
prefix = 'models'

s3_output_job = 's3://{}/{}/{}'.format(job_bucket, prefix, 'comprehend/output')
print(s3_output_job)

s3://sagemaker-experiments-us-east-1-322537213286/models/comprehend/output


In [12]:
iam_role_comprehend_arn = iam_role_comprehend['Role']['Arn']

In [13]:
import datetime
import time

id = str(datetime.datetime.now().strftime("%s"))

training_job = comprehend.create_document_classifier(
    DocumentClassifierName='Amazon-Customer-Reviews-Classifier-'+ id,
    DataAccessRoleArn=iam_role_comprehend_arn,
    InputDataConfig={
        'S3Uri': noheader_train_s3_uri
    },
    OutputDataConfig={
        'S3Uri': s3_output_job
    },
    LanguageCode='en'
)

time.sleep(30)

In [ ]:
import time

jobArn = training_job['DocumentClassifierArn']

max_time = time.time() + 3 * 60 * 60 # 3 hours
while time.time() < max_time:
    describe_custom_classifier = comprehend.describe_document_classifier(
        DocumentClassifierArn = jobArn
    )
    status = describe_custom_classifier["DocumentClassifierProperties"]["Status"]
    print("Custom classifier: {}".format(status))
    
    if status == "TRAINED" or status == "IN_ERROR":
        print('')
        print('Status {}'.format(status))
        print('')
        print(describe_custom_classifier["DocumentClassifierProperties"])
        break
        
    time.sleep(10)

Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: SUBMITTED
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom cl

Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING
Custom classifier: TRAINING


<h2><span style="color:red">위 Classifier가 완료되기 전까지 기다려 주시기 바랍니다.</span></h2>

# Show Results of the Classifier

In [48]:
print(describe_custom_classifier["DocumentClassifierProperties"])

{'DocumentClassifierArn': 'arn:aws:comprehend:us-east-1:322537213286:document-classifier/Amazon-Customer-Reviews-Classifier-1605528492', 'LanguageCode': 'en', 'Status': 'TRAINED', 'SubmitTime': datetime.datetime(2020, 11, 16, 12, 8, 13, 23000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2020, 11, 16, 14, 29, 17, 130000, tzinfo=tzlocal()), 'TrainingStartTime': datetime.datetime(2020, 11, 16, 12, 12, 49, 249000, tzinfo=tzlocal()), 'TrainingEndTime': datetime.datetime(2020, 11, 16, 14, 27, 33, 398000, tzinfo=tzlocal()), 'InputDataConfig': {'DataFormat': 'COMPREHEND_CSV', 'S3Uri': 's3://sagemaker-us-east-1-322537213286/data/amazon_reviews_us_Digital_Software_v1_00_noheader.csv'}, 'OutputDataConfig': {'S3Uri': 's3://sagemaker-experiments-us-east-1-322537213286/models/comprehend/output/322537213286-CLR-5a2fc12a2752585c177294f996eab445/output/output.tar.gz'}, 'ClassifierMetadata': {'NumberOfLabels': 5, 'NumberOfTrainedDocuments': 59288, 'NumberOfTestDocuments': 6587, 'EvaluationMetrics': 

In [49]:
model_arn = describe_custom_classifier["DocumentClassifierProperties"]["DocumentClassifierArn"]
print(model_arn)

arn:aws:comprehend:us-east-1:322537213286:document-classifier/Amazon-Customer-Reviews-Classifier-1605528492


In [50]:
import os
#Retrieve the S3URI from the model output and create jobkey variable.
job_output = describe_custom_classifier["DocumentClassifierProperties"]["OutputDataConfig"]["S3Uri"]
print(job_output)

path_prefix = 's3://{}/'.format(job_bucket)

job_key = os.path.relpath(job_output, path_prefix)

print(job_key)

s3://sagemaker-experiments-us-east-1-322537213286/models/comprehend/output/322537213286-CLR-5a2fc12a2752585c177294f996eab445/output/output.tar.gz
models/comprehend/output/322537213286-CLR-5a2fc12a2752585c177294f996eab445/output/output.tar.gz


In [51]:
#Download the model metrics
s3 = boto3.resource('s3')

s3.Bucket(job_bucket).download_file(job_key, './output.tar.gz')

In [52]:
#Unpack the gzip file
!tar xvzf ./output.tar.gz

output/
output/confusion_matrix.json


In [53]:
import json

with open('./output/confusion_matrix.json') as json_file:
    data = json.load(json_file)
print(json.dumps(data, indent=2, default=str))

{
  "confusion_matrix": [
    [
      885,
      335,
      62,
      24,
      12
    ],
    [
      290,
      719,
      218,
      73,
      17
    ],
    [
      88,
      434,
      490,
      247,
      58
    ],
    [
      17,
      99,
      196,
      704,
      302
    ],
    [
      18,
      34,
      72,
      291,
      902
    ]
  ],
  "labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ],
  "type": "multi_class",
  "all_labels": [
    "1",
    "2",
    "3",
    "4",
    "5"
  ]
}


In [54]:
!{sys.executable} -m pip install tabulate

In [55]:
from IPython.display import HTML, display
import tabulate
table = [['', '1', '2', '3', '4', '5', '(Predicted)'],
         ['1', data['confusion_matrix'][0][0], data['confusion_matrix'][0][1], data['confusion_matrix'][0][2], data['confusion_matrix'][0][3], data['confusion_matrix'][0][4]],
         ['2', data['confusion_matrix'][1][0], data['confusion_matrix'][1][1], data['confusion_matrix'][1][2], data['confusion_matrix'][1][3], data['confusion_matrix'][1][4]],
         ['3', data['confusion_matrix'][2][0], data['confusion_matrix'][2][1], data['confusion_matrix'][2][2], data['confusion_matrix'][2][3], data['confusion_matrix'][2][4]],
         ['4', data['confusion_matrix'][3][0], data['confusion_matrix'][3][1], data['confusion_matrix'][3][2], data['confusion_matrix'][3][3], data['confusion_matrix'][3][4]],
         ['5', data['confusion_matrix'][4][0], data['confusion_matrix'][4][1], data['confusion_matrix'][4][2], data['confusion_matrix'][4][3], data['confusion_matrix'][4][4]],
         ['(Actual)']]
display(HTML(tabulate.tabulate(table, tablefmt='html')))

,1,2,3,4,5,(Predicted)
1,885,335,62,24,12,
2,290,719,218,73,17,
3,88,434,490,247,58,
4,17,99,196,704,302,
5,18,34,72,291,902,
(Actual),,,,,,


# Deploy Endpoint

In [56]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

comprehend_endpoint_name = 'comprehend-inference-ep-' + timestamp_suffix

inference_endpoint_response = comprehend.create_endpoint(
    EndpointName=comprehend_endpoint_name,
    ModelArn = model_arn,
    DesiredInferenceUnits = 1
)

In [57]:
comprehend_endpoint_arn = inference_endpoint_response["EndpointArn"]
print(comprehend_endpoint_arn)

arn:aws:comprehend:us-east-1:322537213286:document-classifier-endpoint/comprehend-inference-ep-17-00-59-31


# Pass Variables to the Next Notebook(s)

In [58]:
%store comprehend_endpoint_arn

Stored 'comprehend_endpoint_arn' (str)


# Predict with Endpoint

In [59]:
describe_response = comprehend.describe_endpoint(
    EndpointArn = comprehend_endpoint_arn
)
print(describe_response)

{'EndpointProperties': {'EndpointArn': 'arn:aws:comprehend:us-east-1:322537213286:document-classifier-endpoint/comprehend-inference-ep-17-00-59-31', 'Status': 'CREATING', 'ModelArn': 'arn:aws:comprehend:us-east-1:322537213286:document-classifier/Amazon-Customer-Reviews-Classifier-1605528492', 'DesiredInferenceUnits': 1, 'CurrentInferenceUnits': 0, 'CreationTime': datetime.datetime(2020, 11, 17, 0, 59, 31, 342000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 11, 17, 0, 59, 31, 342000, tzinfo=tzlocal())}, 'ResponseMetadata': {'RequestId': '20361a7e-5cfb-4034-a226-2607cb0ae664', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '20361a7e-5cfb-4034-a226-2607cb0ae664', 'content-type': 'application/x-amz-json-1.1', 'content-length': '408', 'date': 'Tue, 17 Nov 2020 00:59:31 GMT'}, 'RetryAttempts': 0}}


In [69]:
import time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_response = comprehend.describe_endpoint(
        EndpointArn = comprehend_endpoint_arn
    )
    status = describe_response["EndpointProperties"]["Status"]
    print("Endpoint: {}".format(status))
    
    if status == "IN_SERVICE" or status == "IN_ERROR":
        break
        
    time.sleep(5)

Endpoint: IN_SERVICE


In [70]:
txt = """I loved it!  I will recommend this to everyone."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "5",
      "Score": 0.9751732349395752
    },
    {
      "Name": "4",
      "Score": 0.02274911105632782
    },
    {
      "Name": "3",
      "Score": 0.0016448814421892166
    }
  ],
  "ResponseMetadata": {
    "RequestId": "2b8180f1-0c72-4436-a548-54b7263e268d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "2b8180f1-0c72-4436-a548-54b7263e268d",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "137",
      "date": "Tue, 17 Nov 2020 02:07:42 GMT"
    },
    "RetryAttempts": 0
  }
}


In [71]:
txt = """It's OK."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "3",
      "Score": 0.7971046566963196
    },
    {
      "Name": "2",
      "Score": 0.11186471581459045
    },
    {
      "Name": "4",
      "Score": 0.073480986058712
    }
  ],
  "ResponseMetadata": {
    "RequestId": "8fc43151-31c1-4e74-93d1-057374647c25",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "8fc43151-31c1-4e74-93d1-057374647c25",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "133",
      "date": "Tue, 17 Nov 2020 02:07:43 GMT"
    },
    "RetryAttempts": 0
  }
}


In [72]:
txt = """Really bad.  I hope they don't make this anymore."""

response = comprehend.classify_document(
    Text= txt,
    EndpointArn = comprehend_endpoint_arn
)

import json
print(json.dumps(response, indent=2, default=str))

{
  "Classes": [
    {
      "Name": "1",
      "Score": 0.8331135511398315
    },
    {
      "Name": "2",
      "Score": 0.15223218500614166
    },
    {
      "Name": "3",
      "Score": 0.013668833300471306
    }
  ],
  "ResponseMetadata": {
    "RequestId": "3f199f3c-bd53-4ee7-8656-3ec754b959b5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "3f199f3c-bd53-4ee7-8656-3ec754b959b5",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "136",
      "date": "Tue, 17 Nov 2020 02:07:43 GMT"
    },
    "RetryAttempts": 0
  }
}


# Navigate to Comprehend in the AWS Console
![Comprehend Console](img/comprehend-console.png)

In [73]:
# comprehend.delete_endpoint(
#     EndpointArn=comprehend_endpoint_arn
# )

{'ResponseMetadata': {'RequestId': 'e28cab91-4ce8-48dc-9a80-af31eb94aaa7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e28cab91-4ce8-48dc-9a80-af31eb94aaa7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'date': 'Tue, 17 Nov 2020 02:08:16 GMT'},
  'RetryAttempts': 0}}